# Computer Assignment 3 (Game)

## Implementation

In [2]:
import turtle
import math
import random
from time import sleep
from copy import deepcopy
from enum import Enum
from typing import Union
from sys import argv
from timeit import default_timer as timer

In [3]:
VERTICES_COUNT = 6
SLEEP_TIME = 1.5

Line = tuple[int, int]

In [4]:
class Player(Enum):
    RED = 'red'
    BLUE = 'blue'

    def __invert__(self) -> 'Player':
        return Player.RED if self == Player.BLUE else Player.BLUE

`Player` class is an Enum, which is used to prevent code duplication from `Sim` class.

In [5]:
class MinimaxType(Enum):
    MIN = 0
    MAX = 1

    def __invert__(self) -> 'MinimaxType':
        return MinimaxType.MIN if self == MinimaxType.MAX else MinimaxType.MAX

In [6]:
def winner(player_moves: dict[Player, list[Line]]) -> tuple[Union[Player, None], Union[tuple[Line, Line, Line], None]]:
    for color, moves in player_moves.items():
        if len(moves) < 3:
            continue
        for i in range(0, len(moves) - 2):
            for j in range(i + 1, len(moves) - 1):
                for k in range(j + 1, len(moves)):
                    if len(set(moves[i] + moves[j] + moves[k])) == 3:
                        return ~color, (moves[i], moves[j], moves[k])
    return None, None

The above function will check each triple of lines and calculates the number of points that the triple contains. If the triple contains 3 points, then it will be considered as a triangle and that played will lose the game. Otherwise, the game will continue.

In [18]:
class SimGui:
    Dot = tuple[float, float]

    def __init__(self, title: str, width: int, height: int, vertices_count: int):
        self._screen = turtle.Screen()
        self._screen.setup(width, height)
        self._screen.title(title)
        self._screen.bgcolor(0.117, 0.117, 0.117)
        self._screen.setworldcoordinates(-1.5, -1.5, 1.5, 1.5)
        self._screen.tracer(0, 0)
        turtle.hideturtle()
        self._vertices_count = vertices_count
        self._gen_dots()
        self._draw_board()

    def _gen_dots(self) -> None:
        self._dots = []
        for angle in range(0, 360, 360 // self._vertices_count):
            self._dots.append((math.cos(math.radians(angle)),
                               math.sin(math.radians(angle))))

    def _draw_dot(self, x: float, y: float, color: str) -> None:
        turtle.up()
        turtle.goto(x, y)
        turtle.color(color)
        turtle.dot(15)

    def _draw_line(self, p1: Dot, p2: Dot, color: str, pensize: int = 6) -> None:
        turtle.up()
        turtle.pensize(pensize)
        turtle.goto(p1)
        turtle.down()
        turtle.color(color)
        turtle.goto(p2)

    def _draw_board(self) -> None:
        for dot in self._dots:
            self._draw_dot(dot[0], dot[1], 'dark gray')

    def _lineToDots(self, line: Line) -> tuple[Dot, Dot]:
        return ((math.cos(math.radians(line[0] * 360 // self._vertices_count)),
                 math.sin(math.radians(line[0] * 360 // self._vertices_count))),
                (math.cos(math.radians(line[1] * 360 // self._vertices_count)),
                 math.sin(math.radians(line[1] * 360 // self._vertices_count))))

    def draw(self, player_moves: dict[Player, list[Line]] = {}) -> None:
        turtle.clear()
        self._draw_board()
        for color, moves in player_moves.items():
            for move in moves:
                self._draw_line(*self._lineToDots(move), color.value)
        self._screen.update()

    def show_triangle(self, triangle: tuple[Line, Line, Line]) -> None:
        for line in triangle:
            self._draw_line(*self._lineToDots(line), 'white', 3)
        self._screen.update()

    def close(self) -> None:
        self._screen.bye()

The `show_triangle` function will show the triangle which made the player lose the game. It will make the triangle white.

In [8]:
class MinimaxNode:
    _type: MinimaxType
    _value: float
    _children: dict[Line, 'MinimaxNode']
    _parent: Union['MinimaxNode', None]
    _move: Union[Line, None]
    _player: Player
    _depth: int
    _max_depth: int
    _prune: bool
    _alpha: float
    _beta: float
    _available_moves: list[Line]
    _player_moves: dict[Player, list[Line]]

    def __init__(self, parent: Union['MinimaxNode', None] = None, available_moves: list[Line] = [],
                 player_moves: dict[Player, list[Line]] = {}, prune: bool = False, max_depth: int = 1):
        self._parent = parent
        self._move = None
        self._children = {}
        self._value = 0
        self._type = MinimaxType.MAX if not parent else ~parent._type
        self._depth = 0 if not parent else parent._depth + 1
        self._max_depth = parent._max_depth if parent else max_depth
        self._player = Player.RED if not parent else ~parent._player
        self._alpha = -math.inf if not parent else parent._alpha
        self._beta = math.inf if not parent else parent._beta
        self._available_moves = deepcopy(parent._available_moves) if parent else deepcopy(available_moves)
        self._player_moves = {player: deepcopy(moves) for player, moves in parent._player_moves.items()} if parent else \
            {player: deepcopy(moves) for player, moves in player_moves.items()}
        self._prune = parent._prune if parent else prune

    def _evaluate(self) -> float:
        w = winner(self._player_moves)[0]
        if w is not None:
            return math.inf if w == Player.RED else -math.inf
        h = 0
        for move in self._available_moves:
            res = winner({self._player: self._player_moves[self._player] + [move]})[0]
            if res:
                h -= 1 # player is losing
            res = winner({~self._player: self._player_moves[~self._player] + [move]})[0]
            if res:
                h += 1 # player is winning
        return h if self._player == Player.RED else -h

    def _minimax(self) -> tuple[float, int]:
        if winner(self._player_moves)[0] is not None or self._depth == self._max_depth:
            self._value = self._evaluate()
            return self._value, self._depth
        self._value = -math.inf if self._type == MinimaxType.MAX else math.inf
        optimal_depth = 0
        for move in deepcopy(self._available_moves):
            self._available_moves.remove(move)
            self._player_moves[self._player].append(move)
            child = MinimaxNode(self)
            self._children[move] = child
            val, dep = child._minimax()

            if self._type == MinimaxType.MAX:
                if val > self._value:
                    self._value = val
                    self._move = move
                    optimal_depth = dep
                elif val == self._value and dep > optimal_depth:
                    self._move = move
                    optimal_depth = dep
                self._alpha = max(self._alpha, self._value)
            else:
                if val < self._value:
                    self._value = val
                    self._move = move
                    optimal_depth = dep
                elif val == self._value and dep > optimal_depth:
                    self._move = move
                    optimal_depth = dep
                self._beta = min(self._beta, self._value)

            if self._prune and self._alpha >= self._beta:
                break

            self._player_moves[self._player].remove(move)
            self._available_moves.append(move)

        return self._value, optimal_depth

    def get_best_move(self) -> Line:
        self._minimax()
        return self._move  # type: ignore

`MinimaxNode` is a class used in the mini-max algorithm. The algorithm will make a tree.

### Heuristic

As the heuristic, we should calculate the probability of the formation of a triangle by each player. As a result, first initialize the heuristic with 0. Then, for each line in `available_moves`, if the line will make a triangle, then decrease the heuristic by 1. If the other player will make a triangle, then increase the heuristic by 1. Finally, if the current player is *Red*, then return the heuristic, otherwise return the negative of the heuristic. Also, if the winner has been decided in the current state, then return the heuristic as $\pm\infty$ depending on the winner.  

After running the algorithm for multiple times, I understood that the algorithm will not work properly if the maximum height for algorithm is more than 4. This is because for this level, the winner is decided in the leaf nodes of most of the paths. As a result, the algorithm will see that it will lose the game in either case and may choose a path which can make the player lose even in the next move. The reason is that we use mini-max algorithm for the games that the opponent acts smartly. However, in this game, the opponent acts randomly. So, even when the algorithm understands that all the paths will lead to the loss, if it chooses a path that may last longer, then it will have a higher chance of winning. This is why I changed the algorithm to take the loss height into account to choose the path. This change increased the performance of the algorithm.

In [9]:
class Sim:
    _gui: Union[SimGui, None]
    _turn: Player
    _player_moves: dict[Player, list[Line]]
    _available_moves: list[Line]
    _minimax_depth: int
    _prune: bool

    def __init__(self, minimax_depth: int, prune: bool, gui: bool):
        self._prune = prune
        self._minimax_depth = minimax_depth
        self._gui = SimGui('Game of Sim', 800, 800, VERTICES_COUNT) if gui else None

    def _initialize(self) -> None:
        self._available_moves = []
        for i in range(0, VERTICES_COUNT):
            for j in range(i, VERTICES_COUNT):
                if i != j:
                    self._available_moves.append((i, j))
        self._turn = random.choice([Player.RED, Player.BLUE])
        self._player_moves = {player: [] for player in Player}
        if self._gui:
            self._gui.draw()

    def _swap_turn(self) -> None:
        self._turn = ~self._turn

    def _minimax(self) -> tuple[Line, float]:
        root = MinimaxNode(available_moves=self._available_moves, player_moves=self._player_moves,
                           prune=self._prune, max_depth=self._minimax_depth)
        return root.get_best_move(), root._value

    def _enemy_move(self) -> Line:
        return random.choice(self._available_moves)

    def _game_over(self) -> tuple[Union[Player, None], Union[tuple[Line, Line, Line], None]]:
        return winner(self._player_moves)

    def play(self) -> Player:
        self._initialize()
        while True:
            selection = self._minimax()[0] if self._turn == Player.RED else self._enemy_move()
            selection = tuple(sorted(selection))

            if selection in self._player_moves.values():
                raise Exception("Duplicate Move!")

            self._player_moves[self._turn].append(selection)
            self._available_moves.remove(selection)
            self._swap_turn()
            if self._gui:
                self._gui.draw(self._player_moves)
                sleep(SLEEP_TIME)
            res = self._game_over()
            if res[0]:
                if self._gui:
                    self._gui.show_triangle(res[1])  # type: ignore
                    sleep(SLEEP_TIME)
                return res[0]

    def close(self) -> None:
        if self._gui:
            self._gui.close()

In [9]:
def calcWinChanceAndTime(depth: int, prune: bool, test_count: int) -> None:
    game = Sim(minimax_depth=depth, prune=prune, gui=False)
    start = timer()
    result = {p: 0 for p in Player}
    for i in range(test_count):
        print(f"Processing Test {i + 1}/{test_count}", end="\r")
        res = game.play()
        result[res] += 1
    end = timer()
    print(result)
    print(f"Depth: {depth}, Prune: {prune}")
    print(f"Time: {(end - start) / test_count:.4f}s")
    print(f"Win chance: {result[Player.RED] * 100 // test_count}%")
    print()

## Results

In [20]:
game = Sim(minimax_depth=5, prune=True, gui=True)
res = game.play()
print("Winner: ", res)
game.close()

Winner:  Player.RED


In [18]:
calcWinChanceAndTime(1, False, 100)
calcWinChanceAndTime(3, False, 100)
calcWinChanceAndTime(5, False, 50)

{<Player.RED: 'red'>: 99, <Player.BLUE: 'blue'>: 1}
Depth: 1, Prune: False
Time: 0.0047s
Win chance: 99%

{<Player.RED: 'red'>: 99, <Player.BLUE: 'blue'>: 1}
Depth: 3, Prune: False
Time: 0.4792s
Win chance: 99%

{<Player.RED: 'red'>: 50, <Player.BLUE: 'blue'>: 0}
Depth: 5, Prune: False
Time: 45.8823s
Win chance: 100%



As the algorithm takes a long time to run for the maximum height of 5 without pruning, I ran the algorithm 50 times instead of 100 times.

In [19]:
calcWinChanceAndTime(1, True, 100)
calcWinChanceAndTime(3, True, 100)
calcWinChanceAndTime(5, True, 100)
calcWinChanceAndTime(7, True, 100)

{<Player.RED: 'red'>: 99, <Player.BLUE: 'blue'>: 1}
Depth: 1, Prune: True
Time: 0.0047s
Win chance: 99%

{<Player.RED: 'red'>: 98, <Player.BLUE: 'blue'>: 2}
Depth: 3, Prune: True
Time: 0.0894s
Win chance: 98%

{<Player.RED: 'red'>: 100, <Player.BLUE: 'blue'>: 0}
Depth: 5, Prune: True
Time: 1.1692s
Win chance: 100%

{<Player.RED: 'red'>: 100, <Player.BLUE: 'blue'>: 0}
Depth: 7, Prune: True
Time: 12.4892s
Win chance: 100%



As it is shown, the algorithm time is decreased significantly by pruning. Also, the winning rate is over 98% in all cases, which shows the effectiveness of the algorithm and the heuristic.

## Questions

### 1. What does a good heuristic look like? How did you design your heuristic?

A good heuristic must predict the future state of the game well. In this game, we'd better use the probability of the formation of a triangle as the heuristic. As mentioned in the heuristic section, the heuristic is decreased by 1 for each line in the `available_moves` that makes a triangle for the current player and increased by 1 for each line in the `available_moves` that makes a triangle for the other player.  

Another approach can be not taking the probability of the formation of a triangle by the opponent into account. This heuristic will not be as good as the previous one.

Another approach is to calculate the degree of each vertex (the number of lines with the same color that are connected to the vertex). Then, the heuristic will be the sum of the degrees of the vertices. In this case, for two separate lines, the heuristic will be 4, and if the two lines are connected by a vertex, the heuristic will also be 4. However, the second case can lead to the formation of a triangle. So, this heuristic is not good. My heuristic will return 0 and 1 for the mentioned cases, respectively.

### 2. What is the effect of increasing the depth of the search tree? 

As the depth of the search tree increases, the number of nodes which the algorithm will visit increases. As a result, the algorithm will take more time to run. However, the increase in the accuracy can depend on the algorithm. As mentioned in heuristic section, if we take the depth of the loss into account, then the algorithm will be more accurate. Otherwise, the accuracy will decrease as the depth increases.

$$statesCount = \dfrac{(15 - {currentDepth})!}{(15 - {currentDepth} - n)!}$$
$$if\ n = 7\ \&\ currentDepth = 0\ (root)\longrightarrow statesCount = \dfrac{15!}{8!} = 32,432,400$$

### 3. what is the effect in ordering the nodes when pruning the search tree?

When we use pruning, the order of nodes is important. If we visit the nodes in the wrong order, then we may not be able to prune the nodes well. In this problem, we don't use a specific order at first. However, it is better not to change the initial order of the nodes when running the algorithm.